In [91]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets.samples_generator import make_classification
import tensorflow as tf
tf.reset_default_graph()


In [88]:
### 生成数据集 1000个1维样本   正负各半   0-10 label为1   -10--1 label为0
xa = np.random.randint(0,10,size=500)
xb = np.random.randint(-10,-1,size=500)
xa = xa.tolist()
xb = xb.tolist()
xa.extend(xb)
X1 = np.array(xa)

y = [1]*500
yb = [0]*500

y.extend(yb)
y1 = np.array(y)

print(X1.shape)


state = np.random.get_state()
np.random.shuffle(X1)
np.random.set_state(state)
np.random.shuffle(y1)

X1 = X1.reshape(-1,1)

# print(X1)
print(X1.astype)
print(y1.astype)

(1000,)
<built-in method astype of numpy.ndarray object at 0x2b2bc3f80e90>
<built-in method astype of numpy.ndarray object at 0x2b2bc3ec9670>


In [89]:
### 参数设置
learning_rate = 0.01
training_epochs = 10
batch_size = 10

x = tf.placeholder(tf.float32, [None, 1],name='inputx') # 6 features
y = tf.placeholder(tf.float32, [None, 2]) # 3 classes

W = tf.Variable(tf.zeros([1, 2]))
b = tf.Variable(tf.zeros([2]))

# softmax回归
pred = tf.nn.softmax(tf.matmul(x, W) + b, name="softmax") 
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

prediction_labels = tf.argmax(pred, axis=1, name="output")

init = tf.global_variables_initializer()

In [90]:
### 查看变量名
print(x.name)

inputx_1:0


In [86]:
### label onhot
sess = tf.Session()
sess.run(init)
y2 = tf.one_hot(y1, 2)
y2 = sess.run(y2)

print(y2[:10])

[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [80]:
### 训练
for epoch in range(training_epochs):
    
    _, c, pre = sess.run([optimizer, cost, pred], feed_dict={x: X1, y: y2})
    if (epoch+1) % 2 == 0:
        print(pre[:10], y2[:10])
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    
print ("优化完毕!")
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y2, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
acc = sess.run(accuracy, feed_dict={x: X1, y: y2})
print (acc)

[[ 0.52383691  0.47616306]
 [ 0.45243427  0.54756576]
 [ 0.52383691  0.47616306]
 [ 0.59427822  0.40572172]
 [ 0.45243427  0.54756576]
 [ 0.47616306  0.52383691]
 [ 0.5         0.5       ]
 [ 0.48807478  0.51192528]
 [ 0.53572154  0.46427846]
 [ 0.58272505  0.41727498]] [[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
Epoch: 0002 cost= 0.588380873
[[ 0.56066394  0.43933609]
 [ 0.38024566  0.61975431]
 [ 0.56066394  0.43933609]
 [ 0.7263574   0.27364257]
 [ 0.38024566  0.61975431]
 [ 0.43920606  0.56079394]
 [ 0.49993402  0.50006598]
 [ 0.469457    0.53054303]
 [ 0.5904721   0.4095279 ]
 [ 0.70143884  0.29856113]] [[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
Epoch: 0004 cost= 0.459417999
[[ 0.58764702  0.41235301]
 [ 0.32944965  0.67055035]
 [ 0.58764702  0.41235301]
 [ 0.80520803  0.19479196]
 [ 0.32944965  0.67055035]
 [ 0.41200054  0.5879994 ]
 [ 0.49981818 

In [81]:
### 模型存储为pb格式

graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, ["inputx","output"])
tf.train.write_graph(graph, '.', '0631demo_rf.pb', as_text=False)



INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


'./0631demo_rf.pb'